# [**Comparaison des modèles de forêts aléatoires et d'amplification de gradient basés sur des histogrammes**](https://nbviewer.org/github/Franck-PepperLabs/pepper_dsia_skl_doc_fr/blob/main/docs/examples/1_11_ensembles/plot_forest_hist_grad_boosting_comparison.ipynb)<br/>([_Comparing Random Forests and Histogram Gradient Boosting models_](https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_hist_grad_boosting_comparison.html))

Dans cet exemple, nous comparons les performances des modèles de forêts aléatoires (RF) et d'amplification de gradient basés sur des histogrammes (HGBT) en termes de score et de temps de calcul pour un ensemble de données de régression, bien que **tous les concepts présentés ici s'appliquent également à la classification**.

La comparaison est effectuée en faisant varier les paramètres qui contrôlent le nombre d'arbres selon chaque estimateur :
- `n_estimators` contrôle le nombre d'arbres dans la forêt. C'est un nombre fixe.
- `max_iter` est le nombre maximum d'itérations dans un modèle basé sur l'amplification de gradient. Le nombre d'itérations correspond au nombre d'arbres pour les problèmes de régression et de classification binaire. De plus, le nombre réel d'arbres requis par le modèle dépend des critères d'arrêt.

HGBT utilise l'amplification de gradient pour améliorer itérativement les performances du modèle en ajustant chaque arbre au gradient négatif de la fonction de perte par rapport à la valeur prédite. En revanche, les forêts aléatoires sont basées sur le bagging et utilisent un vote à la majorité pour prédire le résultat.

Pour plus d'informations sur les modèles d'ensemble, consultez le [**Guide de l'utilisateur** (1.11)](https://scikit-learn.org/stable/modules/ensemble.html#ensemble).

In [1]:
# Author:  Arturo Amor <david-arturo.amor-quiroz@inria.fr>
# License: BSD 3 clause

# Chargement de l'ensemble de données

In [2]:
from sklearn.datasets import fetch_california_housing

X, y = fetch_california_housing(return_X_y=True, as_frame=True)
n_samples, n_features = X.shape

HGBT utilise un algorithme basé sur des histogrammes pour des valeurs de caractéristiques binées qui peuvent gérer efficacement de grands ensembles de données (des dizaines de milliers d'échantillons ou plus) avec un grand nombre de caractéristiques (voir [**Pourquoi c'est plus rapide** (1.11.1.1.8)](https://scikit-learn.org/stable/modules/ensemble.html#why-it-s-faster)). L'implémentation de RF dans scikit-learn n'utilise pas le binning et repose sur des divisions exactes, ce qui peut être coûteux en termes de calcul.

In [3]:
print(f"The dataset consists of {n_samples} samples and {n_features} features")

The dataset consists of 20640 samples and 8 features


# Calcul des scores et des temps de calcul

Remarquez que de nombreuses parties de l'implémentation de [**`HistGradientBoostingClassifier`**](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html#sklearn.ensemble.HistGradientBoostingClassifier) et de [**`HistGradientBoostingRegressor`**](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html#sklearn.ensemble.HistGradientBoostingRegressor) sont par défaut parallélisées.

L'implémentation de [**`RandomForestRegressor`**](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor) et de [**`RandomForestClassifier`**](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier) peut également être exécutée sur plusieurs cœurs en utilisant le paramètre `n_jobs`, ici réglé pour correspondre au nombre de cœurs physiques sur la machine hôte. Consultez [**Parallélisme** (8.3.1)](https://scikit-learn.org/stable/computing/parallelism.html#parallelism) pour plus d'informations.

In [4]:
import joblib

N_CORES = joblib.cpu_count(only_physical_cores=True)
print(f"Number of physical cores: {N_CORES}")

Number of physical cores: 6


Contrairement à RF, les modèles HGBT offrent une option d'arrêt anticipé (voir [**Arrêt anticipé de l'amplification de gradient**](https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_early_stopping.html#sphx-glr-auto-examples-ensemble-plot-gradient-boosting-early-stopping-py)) pour éviter d'ajouter de nouveaux arbres inutiles. En interne, l'algorithme utilise un ensemble hors échantillon pour calculer les performances de généralisation du modèle à chaque ajout d'un arbre. Ainsi, si les performances de généralisation n'augmentent pas pendant plus de `n_iter_no_change` itérations, il cesse d'ajouter des arbres.

Les autres paramètres des deux modèles ont été réglés, mais la procédure n'est pas montrée ici pour simplifier l'exemple.

In [5]:
import pandas as pd

from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold

models = {
    "Random Forest": RandomForestRegressor(
        min_samples_leaf=5, random_state=0, n_jobs=N_CORES
    ),
    "Hist Gradient Boosting": HistGradientBoostingRegressor(
        max_leaf_nodes=15, random_state=0, early_stopping=False
    ),
}
param_grids = {
    "Random Forest": {"n_estimators": [10, 20, 50, 100]},
    "Hist Gradient Boosting": {"max_iter": [10, 20, 50, 100, 300, 500]},
}
cv = KFold(n_splits=4, shuffle=True, random_state=0)

results = []
for name, model in models.items():
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grids[name],
        return_train_score=True,
        cv=cv,
    ).fit(X, y)
    result = {"model": name, "cv_results": pd.DataFrame(grid_search.cv_results_)}
    results.append(result)

> **Remarque :** Régler `n_estimators` pour RF résulte généralement en une perte de puissance de calcul. En pratique, il suffit de s'assurer qu'il est suffisamment grand pour que doubler sa valeur ne conduise pas à une amélioration significative du score de test.

# Tracé des résultats

Nous pouvons utiliser un [**`plotly.express.scatter`**](https://plotly.com/python-api-reference/generated/plotly.express.scatter.html) pour visualiser le compromis entre le temps de calcul écoulé et le score moyen au test. Le passage du curseur sur un point donné affiche les paramètres correspondants. Les barres d'erreur correspondent à un écart-type tel que calculé dans les différentes validations croisées.

In [6]:
import plotly.colors as colors
import plotly.express as px
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=1,
    cols=2,
    shared_yaxes=True,
    subplot_titles=["Train time vs score", "Predict time vs score"],
)
model_names = [result["model"] for result in results]
colors_list = colors.qualitative.Plotly * (
    len(model_names) // len(colors.qualitative.Plotly) + 1
)

for idx, result in enumerate(results):
    cv_results = result["cv_results"].round(3)
    model_name = result["model"]
    param_name = list(param_grids[model_name].keys())[0]
    cv_results[param_name] = cv_results[f"param_{param_name}"]
    cv_results["model"] = model_name

    scatter_fig = px.scatter(
        cv_results,
        x="mean_fit_time",
        y="mean_test_score",
        error_x="std_fit_time",
        error_y="std_test_score",
        hover_data=param_name,
        color="model",
    )
    line_fig = px.line(
        cv_results,
        x="mean_fit_time",
        y="mean_test_score",
    )

    scatter_trace = scatter_fig["data"][0]
    line_trace = line_fig["data"][0]
    scatter_trace.update(marker=dict(color=colors_list[idx]))
    line_trace.update(line=dict(color=colors_list[idx]))
    fig.add_trace(scatter_trace, row=1, col=1)
    fig.add_trace(line_trace, row=1, col=1)

    scatter_fig = px.scatter(
        cv_results,
        x="mean_score_time",
        y="mean_test_score",
        error_x="std_score_time",
        error_y="std_test_score",
        hover_data=param_name,
    )
    line_fig = px.line(
        cv_results,
        x="mean_score_time",
        y="mean_test_score",
    )

    scatter_trace = scatter_fig["data"][0]
    line_trace = line_fig["data"][0]
    scatter_trace.update(marker=dict(color=colors_list[idx]))
    line_trace.update(line=dict(color=colors_list[idx]))
    fig.add_trace(scatter_trace, row=1, col=2)
    fig.add_trace(line_trace, row=1, col=2)

fig.update_layout(
    xaxis=dict(title="Train time (s) - lower is better"),
    yaxis=dict(title="Test R2 score - higher is better"),
    xaxis2=dict(title="Predict time (s) - lower is better"),
    legend=dict(x=0.72, y=0.05, traceorder="normal", borderwidth=1),
    title=dict(x=0.5, text="Speed-score trade-off of tree-based ensembles"),
)

: 

Les modèles HGBT et RF s'améliorent tous deux en augmentant le nombre d'arbres dans l'ensemble. Cependant, les scores atteignent un plateau où l'ajout de nouveaux arbres ne fait qu'augmenter le temps d'ajustement et de notation. Le modèle RF atteint ce plateau plus tôt et ne peut jamais atteindre le score de test du modèle HGBDT le plus important.

Notez que les résultats affichés sur le graphique ci-dessus peuvent varier légèrement d'une exécution à l'autre, et encore plus significativement lorsqu'ils sont exécutés sur d'autres machines : essayez d'exécuter cet exemple sur votre propre machine locale.

Dans l'ensemble, on devrait souvent constater que les modèles d'amplification de gradient basés sur des histogrammes dominent uniformément les modèles de forêts aléatoires dans le "compromis score de test vs vitesse d'entraînement" (la courbe HGBDT devrait se trouver en haut à gauche de la courbe RF, sans jamais la croiser). Le "compromis score de test vs vitesse de prédiction" peut également être plus controversé, mais il est le plus souvent favorable à HGBDT. Il est toujours judicieux de vérifier les deux types de modèles (avec un réglage des hyperparamètres) et de comparer leurs performances sur votre problème spécifique pour déterminer lequel est le mieux adapté, mais **HGBT offre presque toujours un meilleur compromis entre vitesse et précision que RF**, que ce soit avec les hyperparamètres par défaut ou en tenant compte du coût du réglage des hyperparamètres.

Il existe cependant une exception à cette règle générale : lors de l'entraînement d'un modèle de classification multiclasse avec un grand nombre de classes possibles, HGBDT ajuste en interne un arbre par classe à chaque itération d'amplification, tandis que les arbres utilisés par les modèles RF sont naturellement multiclasse, ce qui devrait améliorer le compromis vitesse de précision des modèles RF dans ce cas.